<a href="https://colab.research.google.com/github/sarathkumar000/Titanic/blob/master/External_data_Local_Files%2C_Drive%2C_Sheets%2C_and_Cloud_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook provides recipes for loading and saving data from external sources.

# Local file system

## Uploading files from your local file system

`files.upload` returns a dictionary of the files which were uploaded.
The dictionary is keyed by the file name and values are the data which were uploaded.

In [0]:
from google.colab import files

uploaded = files.upload()


Saving Data.csv to Data.csv


In [0]:
import pandas as pd
import io

df = pd.read_csv(io.StringIO(uploaded['Data.csv'].decode('cp1252')),)
df

,should i wear uggs without socks?,I wouldn't recommend wearing warm fuzzy shoes without socks.
0,NaN,I don't know what uggs are. But if they are sh...
1,NaN,Only if you want smelly Uggs. Uggs are warm an...
2,NaN,You should always wear socks with uggs.
3,NaN,Eikes?
4,NaN,Yes
...,...,...
2163,NaN,Thinning the herd
2164,NaN,Let the idiots die or become very sick maybe t...
2165,NaN,I haven't heard about that.
2166,NaN,Pure idiocy .... Absolutely no up side to this...


In [0]:
df.rename(columns = {'should i wear uggs without socks?':'Question'}, inplace = True)
s="I wouldn't recommend wearing warm fuzzy shoes without socks."
df.rename(columns = {s:'Answer'}, inplace = True)
df.columns
df.tail()
# why we are going for dropna means while we are removingf digits from text if parser encounters nan it will throw float error
df=df.dropna(subset = ['Answer'])
text=df['Answer']

print(text)

0       I don't know what uggs are. But if they are sh...
1       Only if you want smelly Uggs. Uggs are warm an...
2                 You should always wear socks with uggs.
3                                                  Eikes?
4                                                     Yes
                              ...                        
2163                                    Thinning the herd
2164    Let the idiots die or become very sick maybe t...
2165                          I haven't heard about that.
2166    Pure idiocy .... Absolutely no up side to this...
2167    Kids have WAY too much time on their hands if ...
Name: Answer, Length: 2149, dtype: object


In [0]:


# that float error occurs here only thats why we put try except finnaly blocks 
def removenum(text):
  text=''.join([y for y in text if not y.isdigit()])
  
  return text
# creating dataframe with columns Textbefore,textafter,textchanged
x=pd.DataFrame(columns=['TextBefore','TextAfter','Changed'])
# Here we are copying the answer part to textbefore
x['TextBefore']=text.copy()
# By iterating all the rows we are finding value for textafter by call func with textbefore
for index,row in x.iterrows():
  row['TextAfter']=removenum(row['TextBefore'])

for index,row in x.iterrows():
  row['Changed']=np.where(row['TextBefore']==row['TextAfter'],'yes','no')

# by here we succesfully removed numbers from 
x





,TextBefore,TextAfter,Changed
0,I don't know what uggs are. But if they are sh...,I don't know what uggs are. But if they are sh...,yes
1,Only if you want smelly Uggs. Uggs are warm an...,Only if you want smelly Uggs. Uggs are warm an...,yes
2,You should always wear socks with uggs.,You should always wear socks with uggs.,yes
3,Eikes?,Eikes?,yes
4,Yes,Yes,yes
...,...,...,...
2163,Thinning the herd,Thinning the herd,yes
2164,Let the idiots die or become very sick maybe t...,Let the idiots die or become very sick maybe t...,yes
2165,I haven't heard about that.,I haven't heard about that.,yes
2166,Pure idiocy .... Absolutely no up side to this...,Pure idiocy .... Absolutely no up side to this...,yes


In [0]:
# to find the count of nan 
null_columns=x.columns[x.isnull().any()]
x[null_columns].isnull().sum()

TextBefore      19
TextAfter       19
Changed       2168
dtype: int64

In [0]:
# !pip install regex
import regex as re
def changeExclam(text):
  return changeQues(re.sub(r"(\!)\1+", ' multiExclamation ', text))
def changeQues(text):
  return changeStop(re.sub(r"(\?)\1+", ' multiQuestion ', text))
def changeStop(text):
  text=re.sub(r"(\!)\1+", ' multiStop ', text)
  return text

xpunc=pd.DataFrame(columns=['TextBefore','TextAfter','TextChanged'])
xpunc['TextBefore']=text.copy()
for index,row in xpunc.iterrows():
  row['TextAfter']=changeExclam(row['TextBefore'])
  row['TextChanged']=np.where(row['TextAfter']==row['TextBefore'],'no','yes')
xpunc
for index,row in xpunc[xpunc['TextChanged']=='yes'].head().iterrows():
  print(row['TextBefore']+"->"+row['TextAfter'])



Pancakes!!!  Yay!!
Hey Otis, did you get a chance to check your email yet?  I just wanted to let you know that the owner confirmed your email for you.  Now you don't have to do ANYTHING!!!  Yay!!  Just use the password I sent you and login.
https://askerspick.net/forum/->Pancakes multiExclamation   Yay multiExclamation 
Hey Otis, did you get a chance to check your email yet?  I just wanted to let you know that the owner confirmed your email for you.  Now you don't have to do ANYTHING multiExclamation   Yay multiExclamation   Just use the password I sent you and login.
https://askerspick.net/forum/
It's on it's last leg.  Good to see you sir!! Please join over at askerspick.net we are moving there.->It's on it's last leg.  Good to see you sir multiExclamation  Please join over at askerspick.net we are moving there.
Pancakes!!!  Yay!!
Hey Otis, did you get a chance to check your email yet?  I just wanted to let you know that the owner confirmed your email for you.  Now you don't have to 

In [0]:
# here we are rmoving the punctuations
# string .punctaution contains all !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
# maketrans(str1, str2, str3) 
# str1 : Specifies the list of characters that need to be replaced.
# str2 : Specifies the list of characters with which the characters need to be replaced.
# str3 : Specifies the list of characters that needs to be deleted.
# Returns : Returns the translation table which specifies the conversions that can be used by translate()

import string
translator=str.maketrans('','',string.punctuation)
xremovepunc=pd.DataFrame(columns=['TextBefore','TextAfter','TextChanged'])
xremovepunc['TextBefore']=text.copy()

# translate(table, delstr)
# table : Translate mapping specified to perform translations.
# delstr : The delete string can be specified as optional argument is not mentioned in table.
# Returns : Returns the translation table which specifies the conversions that can be used by translate()
for index,row in xremovepunc.iterrows():
  row['TextAfter']=row['TextBefore'].translate(translator)
  row['TextChanged']=np.where(row['TextAfter']==row['TextBefore'],'no','yes')

xremovepunc



,TextBefore,TextAfter,TextChanged
0,I don't know what uggs are. But if they are sh...,I dont know what uggs are But if they are shoe...,yes
1,Only if you want smelly Uggs. Uggs are warm an...,Only if you want smelly Uggs Uggs are warm and...,yes
2,You should always wear socks with uggs.,You should always wear socks with uggs,yes
3,Eikes?,Eikes,yes
4,Yes,Yes,no
...,...,...,...
2163,Thinning the herd,Thinning the herd,no
2164,Let the idiots die or become very sick maybe t...,Let the idiots die or become very sick maybe t...,no
2165,I haven't heard about that.,I havent heard about that,yes
2166,Pure idiocy .... Absolutely no up side to this...,Pure idiocy Absolutely no up side to this Dr...,yes


In [0]:
contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
                         (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]
# subn()method is similar to sub() and also returns the new string along with the no. of replacements.
def contractions(text):
  patterns=[(re.compile(regex),repli) for (regex,repli) in contraction_patterns]   
  for (pattern,repli) in patterns:
    (text,count)=re.subn(pattern,repli,text)

  return text  
xcontr=pd.DataFrame(columns=['TextBefore','TextAfter','TextChanged']) 
xcontr['TextBefore']=text.copy()
for index,row in xcontr.iterrows():
  row['TextAfter']=contractions(row['TextBefore'])
  row['TextChanged']=np.where(row['TextAfter']==row['TextBefore'] ,'no','yes')
xcontr


,TextBefore,TextAfter,TextChanged
0,I don't know what uggs are. But if they are sh...,I do not know what uggs are. But if they are s...,yes
1,Only if you want smelly Uggs. Uggs are warm an...,Only if you want smelly Uggs. Uggs are warm an...,yes
2,You should always wear socks with uggs.,You should always wear socks with uggs.,no
3,Eikes?,Eikes?,no
4,Yes,Yes,no
...,...,...,...
2163,Thinning the herd,Thinning the herd,no
2164,Let the idiots die or become very sick maybe t...,Let the idiots die or become very sick maybe t...,no
2165,I haven't heard about that.,I have not heard about that.,yes
2166,Pure idiocy .... Absolutely no up side to this...,Pure idiocy .... Absolutely no up side to this...,no


In [0]:
text_lowercase = pd.DataFrame(columns=['TextBefore', 'TextAfter', 'Changed'])
text_lowercase['TextBefore'] = text.copy()


for index,row in text_lowercase.iterrows():
  row['TextAfter']=row['TextBefore'].lower()
  row['Changed']=np.where(row['TextAfter']==row['TextBefore'],'no','yes')
text_lowercase

,TextBefore,TextAfter,Changed
0,I don't know what uggs are. But if they are sh...,i don't know what uggs are. but if they are sh...,yes
1,Only if you want smelly Uggs. Uggs are warm an...,only if you want smelly uggs. uggs are warm an...,yes
2,You should always wear socks with uggs.,you should always wear socks with uggs.,yes
3,Eikes?,eikes?,yes
4,Yes,yes,yes
...,...,...,...
2163,Thinning the herd,thinning the herd,yes
2164,Let the idiots die or become very sick maybe t...,let the idiots die or become very sick maybe t...,yes
2165,I haven't heard about that.,i haven't heard about that.,yes
2166,Pure idiocy .... Absolutely no up side to this...,pure idiocy .... absolutely no up side to this...,yes


In [0]:
def replace(text):
  









def find(tokens):
  words=[]
  i,l=0,len(tokens)
  while i<l:
    word=tokens[i]
    if word=='not' and i+1 < l:
      s=replace(tokens[i+1])
      if s:
        words.appned(s)
        i+=2
        continue
    words.append(word)
    i+=1

  return words




def tokenizer(text):
  tokens=nltk.word_tokenizer(text)
  result=find(tokens)
  return " ".join(result)


xAntorepla=pd.DataFrame(columns=['TextBefore','TextAfter','TextChnaged'])
xAntorepla['TextBefore']=text.copy()
for index,row in xAntorepla.iterrows():
  xAntorepla['TextAfter']=tokenizer(xAntorepla['TextBefore'])
  xAntorepla['TextChanged']=np.where(xAntorepla['TextBefore']==xAntorepla['TextAfter'],'no','yes')

## Downloading files to your local file system

`files.download` will invoke a browser download of the file to your local computer.


In [0]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

# Google Drive

You can access files in Drive in a number of ways, including:
- Mounting your Google Drive in the runtime's virtual machine
- Using a wrapper around the API such as [PyDrive](https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html)
- Using the [native REST API](https://developers.google.com/drive/v3/web/about-sdk)



Examples of each are below.

## Mounting Google Drive locally

The example below shows how to mount your Google Drive on your runtime using an authorization code, and how to write and read files there. Once executed, you will be able to see the new file (`foo.txt`) at [https://drive.google.com/](https://drive.google.com/).

Note this only supports reading, writing, and moving files; to programmatically modify sharing settings or other metadata, use one of the other options below.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code
Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

In [0]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


## PyDrive

The examples below demonstrate authentication and file upload/download using PyDrive. More examples are available in the [PyDrive documentation](https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html).

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authenticate and create the PyDrive client.


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Create and upload a text file.


In [0]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 14vDAdqp7BSCQnoougmgylBexIr2AQx2T


Load a file by ID and print its contents.


In [0]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


## Drive REST API

In order to use the Drive API, we must first authenticate and construct an API client.


In [0]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

With this client, we can use any of the functions in the [Google Drive API reference](https://developers.google.com/drive/v3/reference/). Examples follow.


### Creating a new Drive file with data from Python

First, create a local file to upload.

In [0]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Upload it using the [`files.create`](https://developers.google.com/drive/v3/reference/files/create) method. Further details on uploading files are available in the [developer documentation](https://developers.google.com/drive/v3/web/manage-uploads).

In [0]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1Cw9CqiyU6zbXFD9ViPZu_3yX-sYF4W17


After executing the cell above, you will see a new file named 'Sample file' at [https://drive.google.com/](https://drive.google.com/).

### Downloading data from a Drive file into Python

Download the file we uploaded above.

In [0]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file'


In order to download a different file, set `file_id` above to the ID of that file, which will look like "1uBtlaggVyWshwcyP6kEI-y_W3P8D26sz".

# Google Sheets

Our examples below use the open-source [`gspread`](https://github.com/burnash/gspread) library for interacting with Google Sheets.

First, install the package using `pip`.

In [0]:
!pip install --upgrade --quiet gspread

Import the library, authenticate, and create the interface to Sheets.

In [0]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

Below is a small set of `gspread` examples. Additional examples are availabe at the [`gspread` GitHub page](https://github.com/burnash/gspread#more-examples).

## Creating a new sheet with data from Python

In [0]:
sh = gc.create('My cool spreadsheet')

After executing the cell above, you will see a new spreadsheet named 'My cool spreadsheet' at [https://sheets.google.com](https://sheets.google.com/).

Open our new sheet and add some random data.

In [0]:
worksheet = gc.open('My cool spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1dsQeN0YzXuM387l_CuyEbsYzL2ew9TJFzR-E-RQnwxs',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': 'Sheet1!A1:C2',
 'updatedRows': 2}

## Downloading data from a sheet into Python as a Pandas DataFrame

Read back the random data that we inserted above and convert the result into a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [0]:
worksheet = gc.open('My cool spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
pd.DataFrame.from_records(rows)

[['6', '3', '4'], ['7', '2', '1']]


,0,1,2
0,6,3,4
1,7,2,1


# Google Cloud Storage (GCS)

In order to use Colaboratory with GCS, you'll need to create a [Google Cloud project](https://cloud.google.com/storage/docs/projects) or use a pre-existing one.

Specify your project ID below:

In [0]:
project_id = 'Your_project_ID_here'

Files in GCS are contained in [buckets](https://cloud.google.com/storage/docs/key-terms#buckets).

Buckets must have a globally-unique name, so we generate one here.

In [0]:
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

In order to access GCS, we must authenticate.

In [0]:
from google.colab import auth
auth.authenticate_user()

GCS can be accessed via the `gsutil` command-line utility or via the native Python API.

## `gsutil`

First, we configure `gsutil` to use the project we specified above by using `gcloud`.

In [0]:
!gcloud config set project {project_id}

Updated property [core/project].


Create a local file to upload.

In [0]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Make a bucket to which we'll upload the file ([documentation](https://cloud.google.com/storage/docs/gsutil/commands/mb)).

In [0]:
!gsutil mb gs://{bucket_name}

Creating gs://colab-sample-bucket-44971372-baaf-11e7-ae30-0242ac110002/...


Copy the file to our new bucket ([documentation](https://cloud.google.com/storage/docs/gsutil/commands/cp)).

In [0]:
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/

Copying file:///tmp/to_upload.txt [Content-Type=text/plain]...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       


Dump the contents of our newly copied file to make sure everything worked ([documentation](https://cloud.google.com/storage/docs/gsutil/commands/cat)).


In [0]:
!gsutil cat gs://{bucket_name}/to_upload.txt

my sample file

In [0]:
#@markdown Once the upload has finished, the data will appear in the Cloud Console storage browser for your project:
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


Finally, we'll download the file we just uploaded in the example above. It's as simple as reversing the order in the `gsutil cp` command.

In [0]:
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt
  
# Print the result to make sure the transfer worked.
!cat /tmp/gsutil_download.txt

Copying gs://colab-sample-bucket483f20dc-baaf-11e7-ae30-0242ac110002/to_upload.txt...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       
my sample file

## Python API

These snippets based on [a larger example](https://github.com/GoogleCloudPlatform/storage-file-transfer-json-python/blob/master/chunked_transfer.py) that shows additional uses of the API.

 First, we create the service client.

In [0]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

Create a local file to upload.

In [0]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Create a bucket in the project specified above.

In [0]:
# Use a different globally-unique bucket name from the gsutil example above.
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

body = {
  'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
  'location': 'us',
}
gcs_service.buckets().insert(project=project_id, body=body).execute()
print('Done')

Done


Upload the file to our newly created bucket.

In [0]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/tmp/to_upload.txt', 
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name, 
                                       name='to_upload.txt',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')

Upload complete


In [0]:
#@markdown Once the upload has finished, the data will appear in the Cloud Console storage browser for your project:
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


Download the file we just uploaded.

In [0]:
from apiclient.http import MediaIoBaseDownload

with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

Download complete


Inspect the downloaded file.


In [0]:
!cat /tmp/downloaded_from_gcs.txt

my sample file